In [1]:
import os
import numpy as np

# Function to load dataset from file. This is needed so we can easily load the two datasets without copy pasteing.
def load_data( name ):
    X_train = np.load(os.path.join('data', name, name + '_train_vectors.npy'))
    X_test = np.load(os.path.join('data', name, name + '_test_vectors.npy'))
    Y_train = np.load(os.path.join('data', name, name + '_train_labels.npy'))
    Y_test = np.load(os.path.join('data', name, name + '_test_labels.npy'))

    # The images need to have shape (28, 28, 1), we didn't take care of this in preprocessing.
    X_train = np.expand_dims(X_train, -1)
    X_test = np.expand_dims(X_test, -1)

    return X_train, Y_train, X_test, Y_test

# The same model is used for both datasets so it is more convenient to make them in a funtion.
def make_model(X_train, Y_train, batch_selection, log_file):

    # This is a simple convolutional neural network. It isn't the best possible network for MNIST
    # but the point here is to test how much batch selection methods will speed up a CNN, not the CNN itself.
    model = Sequential()
    model.add(layers.Input(shape = (28, 28, 1,)))
    model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
    model.add(layers.MaxPooling2D( pool_size = (2, 2)))
    model.add(layers.Conv2D(64, kernel_size = (3, 3), activation = "relu"))
    model.add(layers.MaxPooling2D(pool_size = (2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(10, activation = "softmax"))
    
    # Put the model in our custom training loop.
    training = TrainingLoop(
        model = model,
        X = X_train,
        y = Y_train,
        optimizer = keras.optimizers.Adam(),
        loss_function = keras.losses.CategoricalCrossentropy(from_logits=True),
        batch_size = 64,
        train_metrics = tf.keras.metrics.CategoricalAccuracy(),
        val_metrics = tf.keras.metrics.CategoricalAccuracy(),
        validation_split = 0.2,
        batch_selection = batch_selection,
        log_file = "logs/" + log_file,
    )

    # We still have to compile the model for the test evaluation.
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics=["accuracy"])

    return model, training


In [2]:
from loop import TrainingLoop
import tensorflow as tf

# These lines will make the gpu not give errors.
gpus= tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential


In [3]:
from batch_selection import windowed_batch_selector, sorting_batch_selector

selector_list = [[None, 'original'], [windowed_batch_selector, 'windowed'], [sorting_batch_selector, 'sorting']]

X_train, Y_train, X_test, Y_test = load_data( "mnist" )
for selector in selector_list:
    print( "\n\n" + selector[1] + "\n")
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model, training = make_model( X_train, Y_train, selector[0], selector[1] + "/mnist.csv" )

    training.train(epochs = 10)




original

Epoch 1/10	Loss: 1.5121	Metrics: 0.9156: 	Validation metrics: 0.9678: 	100% | 750/750 [00:07<00:00, 106.64it/s]
Epoch 2/10	Loss: 1.4941	Metrics: 0.9713: 	Validation metrics: 0.9716: 	100% | 750/750 [00:04<00:00, 158.73it/s]
Epoch 3/10	Loss: 1.5062	Metrics: 0.9788: 	Validation metrics: 0.9806: 	100% | 750/750 [00:05<00:00, 149.41it/s]
Epoch 4/10	Loss: 1.4925	Metrics: 0.9827: 	Validation metrics: 0.9828: 	100% | 750/750 [00:04<00:00, 154.87it/s]
Epoch 5/10	Loss: 1.4913	Metrics: 0.9850: 	Validation metrics: 0.9835: 	100% | 750/750 [00:04<00:00, 155.77it/s]
Epoch 6/10	Loss: 1.4904	Metrics: 0.9866: 	Validation metrics: 0.9851: 	100% | 750/750 [00:04<00:00, 155.40it/s]
Epoch 7/10	Loss: 1.4796	Metrics: 0.9882: 	Validation metrics: 0.9864: 	100% | 750/750 [00:04<00:00, 167.23it/s]
Epoch 8/10	Loss: 1.4943	Metrics: 0.9893: 	Validation metrics: 0.9850: 	100% | 750/750 [00:04<00:00, 171.87it/s]
Epoch 9/10	Loss: 1.4709	Metrics: 0.9907: 	Validation metrics: 0.9849: 	100% | 750/750 [00:04

In [4]:
X_train, Y_train, X_test, Y_test = load_data( "fashion_mnist" )
for selector in selector_list:
    print( "\n\n" + selector[1] + "\n")
    # Set random seed so the comparison of different solutions won't be affected by it.
    tf.random.set_seed(42)
    np.random.seed(42)
    
    model, training = make_model( X_train, Y_train, selector[0], selector[1] + "/fashion_mnist.csv" )

    training.train(epochs = 20)



original

Epoch 1/20	Loss: 1.7431	Metrics: 0.7360: 	Validation metrics: 0.7993: 	100% | 750/750 [00:05<00:00, 143.87it/s]
Epoch 2/20	Loss: 1.7369	Metrics: 0.8219: 	Validation metrics: 0.8393: 	100% | 750/750 [00:04<00:00, 153.31it/s]
Epoch 3/20	Loss: 1.6985	Metrics: 0.8569: 	Validation metrics: 0.8604: 	100% | 750/750 [00:04<00:00, 152.08it/s]
Epoch 4/20	Loss: 1.6919	Metrics: 0.8704: 	Validation metrics: 0.8699: 	100% | 750/750 [00:05<00:00, 148.50it/s]
Epoch 5/20	Loss: 1.6785	Metrics: 0.8774: 	Validation metrics: 0.8739: 	100% | 750/750 [00:05<00:00, 140.15it/s]
Epoch 6/20	Loss: 1.6866	Metrics: 0.8829: 	Validation metrics: 0.8670: 	100% | 750/750 [00:05<00:00, 143.11it/s]
Epoch 7/20	Loss: 1.6386	Metrics: 0.8885: 	Validation metrics: 0.8800: 	100% | 750/750 [00:05<00:00, 139.82it/s]
Epoch 8/20	Loss: 1.6105	Metrics: 0.8949: 	Validation metrics: 0.8851: 	100% | 750/750 [00:05<00:00, 136.94it/s]
Epoch 9/20	Loss: 1.6044	Metrics: 0.8996: 	Validation metrics: 0.8880: 	100% | 750/750 [00:05